# ニューラルネットワーク入門

## Section 3 Pytorchでの実装

In [ ]:
# # Googleドライブのマウント（Colab使いのみ）

# from google.colab import drive
# drive.mount('/content/drive')

# %cd /content/drive/MyDrive/dlc/week3

In [ ]:
# 図表が使えるようにする

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

### 前処理を定義

In [ ]:
from torchvision import transforms

my_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5, 0.5, 0.5],  # RGB 平均
        [0.5, 0.5, 0.5]   # RGB 標準偏差
    )
])

### データを準備

Dataset : データを一つのデータベースにまとめる

In [ ]:
# MNISTをインポートする

from torchvision import datasets, transforms

train_Dataset = datasets.CIFAR10(
    './data'
    , train=True
    , download=True 
    , transform=my_transform
)

test_dataset = datasets.CIFAR10(
    './data'
    , train=False
    , transform=my_transform
)

In [ ]:
# 学習データと検証データの分割

import torch 

train_ratio = 0.9
train_num = int(len(train_Dataset) * train_ratio)
valid_num = len(train_Dataset) - train_num

train_dataset, valid_dataset = torch.utils.data.random_split(
    train_Dataset
    , [train_num, valid_num]
)

In [ ]:
# 数を確認

print(f'Train : {len(train_dataset)}')
print(f'Valid : {len(valid_dataset)}')
print(f'Test  : {len(test_dataset)}')
print(test_dataset.data.shape)

DataLoader : データをミニバッチ単位で取り出す 

In [ ]:
# ミニバッチごとに取り出す

from torch.utils.data import DataLoader

batch_size = 32

train_loader = DataLoader(
    dataset=train_dataset
    , batch_size=batch_size 
    , shuffle=True
)

valid_loader = DataLoader(
    dataset=valid_dataset
    , batch_size=batch_size 
    , shuffle=True
)

test_loader = DataLoader(
    dataset=test_dataset
    , batch_size=batch_size
    , shuffle=False
)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
            'dog', 'frog', 'horse', 'ship', 'truck')  # CIFAR10のクラス
 
for images, labels in train_loader:
    print(images.shape)  # torch.Size([32, 3, 32, 32])

    for i in range(3):
        for j in range(10):
            image = images[i*10+j] / 2 + 0.5
            image = image.numpy()
            plt.subplot(3, 10, i*10+j + 1)
            plt.imshow(np.transpose(image, (1, 2, 0)))
            
            # 対応するラベル
            plt.title(classes[int(labels[i*10+j])])
            
            # 軸目盛や値はいらないので消す
            plt.tick_params(
                # labelbottom=False,
                labelleft=False,
                labelright=False,
                labeltop=False,
                bottom=False,
                left=False,
                right=False,
                top=False
            )

    plt.show()
    break

### ネットワークを定義

In [ ]:
# ネットワークの定義

import torch.nn as nn
import torch.nn.functional as F

# クラス数
num_classes = 10

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 256)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.fc2(x)
        return x

In [ ]:
# デバイスに転送

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Net().to(device)

In [ ]:
# ネットワークの構造を確認

print(model)

### 損失関数と最適化手法を定義

In [ ]:
# 損失関数と最適化手法を設定

import torch.optim as optim

# 損失関数: 交差エントロピー
criterion = nn.CrossEntropyLoss()

# 最適化手法: Adam
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

### 学習

In [ ]:
# 結果保存用リスト

train_loss_list, train_acc_list, valid_loss_list, valid_acc_list = [], [], [], []

In [ ]:
# エポック数を設定

num_epochs = 10

In [ ]:
###  training
print ('training start ...')

 
for epoch in range(num_epochs):
    train_loss, train_acc, valid_loss, valid_acc = 0, 0, 0, 0
    
    # ======== train_mode ======
    model.train()
    for i, (images, labels) in enumerate(train_loader):  # ミニバッチ回数実行
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # 勾配リセット
        outputs = model(images)  # 順伝播の計算
        loss = criterion(outputs, labels)  # lossの計算
        train_loss += loss.item()  # train_loss に結果を蓄積
        acc = (outputs.max(1)[1] == labels).sum()  #  予測とラベルが合っている数の合計
        train_acc += acc.item()  # train_acc に結果を蓄積
        loss.backward()  # 逆伝播の計算        
        optimizer.step()  # 重みの更新
        avg_train_loss = train_loss / len(train_loader.dataset)  # lossの平均を計算
        avg_train_acc = train_acc / len(train_loader.dataset)  # accの平均を計算

    
    # ======== valid_mode ======
    model.eval()
    with torch.no_grad():  # 必要のない計算を停止
      for images, labels in valid_loader:        
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          loss = criterion(outputs, labels)
          valid_loss += loss.item()
          acc = (outputs.max(1)[1] == labels).sum()
          valid_acc += acc.item()
    avg_valid_loss = valid_loss / len(valid_loader.dataset)
    avg_valid_acc = valid_acc / len(valid_loader.dataset)
    
    # print log
    print ('Epoch [{}/{}], Loss: {loss:.4f}, val_loss: {valid_loss:.4f}, val_acc: {valid_acc:.4f}' 
                   .format(epoch+1, num_epochs, i+1, loss=avg_train_loss, valid_loss=avg_valid_loss, valid_acc=avg_valid_acc))
     
    # append list for polt graph after training
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)
    valid_loss_list.append(avg_valid_loss)
    valid_acc_list.append(avg_valid_acc)
    
# ======== fainal test ======
model.eval()
 

# plot graph 
plt.figure()
plt.plot(range(len(train_loss_list)), train_loss_list, color='blue', linestyle='-', label='train_loss')
plt.plot(range(len(valid_loss_list)), valid_loss_list, color='green', linestyle='--', label='valid_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Training and validation loss')
plt.grid()
 
plt.figure()
plt.plot(range(len(train_acc_list)), train_acc_list, color='blue', linestyle='-', label='train_acc')
plt.plot(range(len(valid_acc_list)), valid_acc_list, color='green', linestyle='--', label='valid_acc')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title('Training and validation accuracy')
plt.grid()

In [ ]:
# 重みの保存

torch.save(model.state_dict(), 'mnist_model_20.ckpt')

In [ ]:
# テストデータでAccを確認

model.eval()
with torch.no_grad():
    total = 0
    test_acc = 0
    for images, labels in test_loader:        
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        test_acc += (outputs.max(1)[1] == labels).sum().item()
        total += labels.size(0)
    print('test_accuracy: {} %'.format(100 * test_acc / total)) 

### 学習済みモデルの利用

In [ ]:
# vgg16

from torchvision import models

model_vgg = models.vgg16(pretrained=True)
vggnet = model_vgg.to(device)

In [ ]:
###  training
print ('training start ...')

 
for epoch in range(num_epochs):
    train_loss, train_acc, valid_loss, valid_acc = 0, 0, 0, 0
    
    # ======== train_mode ======
    vggnet.train()
    for i, (images, labels) in enumerate(train_loader):  # ミニバッチ回数実行
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # 勾配リセット
        outputs = vggnet(images)  # 順伝播の計算
        loss = criterion(outputs, labels)  # lossの計算
        train_loss += loss.item()  # train_loss に結果を蓄積
        acc = (outputs.max(1)[1] == labels).sum()  #  予測とラベルが合っている数の合計
        train_acc += acc.item()  # train_acc に結果を蓄積
        loss.backward()  # 逆伝播の計算        
        optimizer.step()  # 重みの更新
        avg_train_loss = train_loss / len(train_loader.dataset)  # lossの平均を計算
        avg_train_acc = train_acc / len(train_loader.dataset)  # accの平均を計算

    
    # ======== valid_mode ======
    vggnet.eval()
    with torch.no_grad():  # 必要のない計算を停止
      for images, labels in valid_loader:        
          images, labels = images.to(device), labels.to(device)
          outputs = vggnet(images)
          loss = criterion(outputs, labels)
          valid_loss += loss.item()
          acc = (outputs.max(1)[1] == labels).sum()
          valid_acc += acc.item()
    avg_valid_loss = valid_loss / len(valid_loader.dataset)
    avg_valid_acc = valid_acc / len(valid_loader.dataset)
    
    # print log
    print ('Epoch [{}/{}], Loss: {loss:.4f}, val_loss: {valid_loss:.4f}, val_acc: {valid_acc:.4f}' 
                   .format(epoch+1, num_epochs, i+1, loss=avg_train_loss, valid_loss=avg_valid_loss, valid_acc=avg_valid_acc))
     
    # append list for polt graph after training
    train_loss_list.append(avg_train_loss)
    train_acc_list.append(avg_train_acc)
    valid_loss_list.append(avg_valid_loss)
    valid_acc_list.append(avg_valid_acc)
    
# ======== fainal test ======
vggnet.eval()
 

# plot graph 
plt.figure()
plt.plot(range(len(train_loss_list)), train_loss_list, color='blue', linestyle='-', label='train_loss')
plt.plot(range(len(valid_loss_list)), valid_loss_list, color='green', linestyle='--', label='valid_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Training and validation loss')
plt.grid()
 
plt.figure()
plt.plot(range(len(train_acc_list)), train_acc_list, color='blue', linestyle='-', label='train_acc')
plt.plot(range(len(valid_acc_list)), valid_acc_list, color='green', linestyle='--', label='valid_acc')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.title('Training and validation accuracy')
plt.grid()

In [ ]:
# resnet50

from torchvision import models

model_res = models.resnet50(pretrained=True)
model_res.fc = nn.Linear(model_res.fc.in_features, 10)
resnet = model_res.to(device)